In [43]:
import sys
sys.path.append('/Users/marcelbreithaupt/Library/Mobile Documents/com~apple~CloudDocs/ENV/Master/Semester 2/DevOps/Project')

In [62]:
# runcmd: cd ../.. & venv\Scripts\python server/py/dog_template.py
from server.py.game import Game, Player
from typing import List, Optional, ClassVar
from pydantic import BaseModel
from enum import Enum
import random


class Card(BaseModel):
    """
    Base class for all cards.
    """
    suit: str  # card suit (color)
    rank: str  # card rank

    def execute(self, marble, board, distance=None, move_out=False):
        """
        Executes the action of the card.
        This method will be overridden by special card subclasses.
        """
        raise NotImplementedError("Subclasses must implement the execute method.")


class Ace(Card):
    """
    Special Card: Ace
    Rule 1: Marble can move distance 1
    Rule 2: Marble can move distance 11
    """
    def __init__(self):
        super().__init__(suit='♠', rank='A')

    def execute(self, marble, board, distance):
        """Executes the Ace card action: Move a marble 1 or 11 spaces forward"""
        if distance not in (1, 11):
            raise ValueError("Based on the Rules, you can only choose distance 1 or 11.")
        marble.move_forward(board, distance)
        print(f"Ace executed: Marble moved {distance} spaces.")


class King(Card):
    """
    Special Card: King
    Rule 1: Bring a marble out
    Rule 2: Move 13 spaces forward
    """
    def __init__(self):
        super().__init__(suit='♠', rank='K')

    def execute(self, marble, board, move_out=False, color=None):
        """Executes the King card action: Bring a marble into play or move 13 spaces forward"""
        if move_out:
            marble.enter_play(board, color)
        else:
            marble.move_forward(board, 13)
        print(f"King executed: Marble {'brought into play' if move_out else 'moved 13 spaces'}.")

class Queen(Card):
    """
    Special Card: Queen
    Rule: Move a marble 10 spaces backward
    """
    def __init__(self):
        super().__init__(suit='♠', rank='Q')

    def execute(self, marble, board):
        """Executes the Queen card action: Move the marble 10 spaces backward"""
        if not marble.is_in_play:
            raise ValueError("Marble is not in play.")
        marble.position = board.calculate_new_position(marble.position, -10)
        print(f"Queen executed: Marble moved backward to position {marble.position}.")


class Marble(BaseModel):
    """
    Represents a marble on the board.
    """
    pos: int       # position on board (0 to 95)
    is_save: bool  # true if marble was moved out of kennel and was not yet moved

    def move_forward(self, board, distance):
        """
        Moves the marble forward by the given distance on the board.
        """
        if not self.is_save:
            raise ValueError("Marble is not in play.")
        self.pos = board.calculate_new_position(self.pos, distance)
        print(f"Marble moved to position {self.pos}.")

    def enter_play(self, board, color):
        """
        Brings the marble into play.
        """
        if self.is_save:
            raise ValueError("Marble is already in play.")
        self.is_save = True
        self.pos = board.get_starting_position(color)
        print(f"Marble entered play at position {self.pos}.")


class PlayerState(BaseModel):
    name: str                  # name of player
    list_card: List[Card]      # list of cards
    list_marble: List[Marble]  # list of marbles


class Action(BaseModel):
    card: Card                 # card to play
    pos_from: Optional[int]    # position to move the marble from
    pos_to: Optional[int]      # position to move the marble to
    card_swap: Optional[Card]  # optional card to swap ()


class GamePhase(str, Enum):
    SETUP = 'setup'            # before the game has started
    RUNNING = 'running'        # while the game is running
    FINISHED = 'finished'      # when the game is finished


class GameState(BaseModel):

    LIST_SUIT: ClassVar[List[str]] = ['♠', '♥', '♦', '♣']  # 4 suits (colors)
    LIST_RANK: ClassVar[List[str]] = [
        '2', '3', '4', '5', '6', '7', '8', '9', '10',      # 13 ranks + Joker
        'J', 'Q', 'K', 'A', 'JKR'
    ]
    LIST_CARD: ClassVar[List[Card]] = [
        # 2: Move 2 spots forward
        Card(suit='♠', rank='2'), Card(suit='♥', rank='2'), Card(suit='♦', rank='2'), Card(suit='♣', rank='2'),
        # 3: Move 3 spots forward
        Card(suit='♠', rank='3'), Card(suit='♥', rank='3'), Card(suit='♦', rank='3'), Card(suit='♣', rank='3'),
        # 4: Move 4 spots forward or back
        Card(suit='♠', rank='4'), Card(suit='♥', rank='4'), Card(suit='♦', rank='4'), Card(suit='♣', rank='4'),
        # 5: Move 5 spots forward
        Card(suit='♠', rank='5'), Card(suit='♥', rank='5'), Card(suit='♦', rank='5'), Card(suit='♣', rank='5'),
        # 6: Move 6 spots forward
        Card(suit='♠', rank='6'), Card(suit='♥', rank='6'), Card(suit='♦', rank='6'), Card(suit='♣', rank='6'),
        # 7: Move 7 single steps forward
        Card(suit='♠', rank='7'), Card(suit='♥', rank='7'), Card(suit='♦', rank='7'), Card(suit='♣', rank='7'),
        # 8: Move 8 spots forward
        Card(suit='♠', rank='8'), Card(suit='♥', rank='8'), Card(suit='♦', rank='8'), Card(suit='♣', rank='8'),
        # 9: Move 9 spots forward
        Card(suit='♠', rank='9'), Card(suit='♥', rank='9'), Card(suit='♦', rank='9'), Card(suit='♣', rank='9'),
        # 10: Move 10 spots forward
        Card(suit='♠', rank='10'), Card(suit='♥', rank='10'), Card(suit='♦', rank='10'), Card(suit='♣', rank='10'),
        # Jake: A marble must be exchanged
        Card(suit='♠', rank='J'), Card(suit='♥', rank='J'), Card(suit='♦', rank='J'), Card(suit='♣', rank='J'),
        # Queen: Move 12 spots forward
        Card(suit='♠', rank='Q'), Card(suit='♥', rank='Q'), Card(suit='♦', rank='Q'), Card(suit='♣', rank='Q'),
        # King: Start or move 13 spots forward
        Card(suit='♠', rank='K'), Card(suit='♥', rank='K'), Card(suit='♦', rank='K'), Card(suit='♣', rank='K'),
        # Ass: Start or move 1 or 11 spots forward
        Card(suit='♠', rank='A'), Card(suit='♥', rank='A'), Card(suit='♦', rank='A'), Card(suit='♣', rank='A'),
        # Joker: Use as any other card you want
        Card(suit='', rank='JKR'), Card(suit='', rank='JKR'), Card(suit='', rank='JKR')
    ] * 2

    cnt_player: int = 4                # number of players (must be 4)
    phase: GamePhase                   # current phase of the game
    cnt_round: int                     # current round
    bool_game_finished: bool           # true if game has finished
    bool_card_exchanged: bool          # true if cards was exchanged in round
    idx_player_started: int            # index of player that started the round
    idx_player_active: int             # index of active player in round
    list_player: List[PlayerState]     # list of players
    list_id_card_draw: List[Card]      # list of cards to draw
    list_id_card_discard: List[Card]   # list of cards discarded
    card_active: Optional[Card]        # active card (for 7 and JKR with sequence of actions)


class Dog(Game):
    def __init__(self) -> None:
        """ Game initialization (set_state call not necessary, we expect 4 players) """
        self.board_numbers = range(0, 96)
        self.board = {
            "blue": {"home": self.board_numbers[64:68], "start": self.board_numbers[0], "finish": self.board_numbers[68:72]},
            "green": {"home": self.board_numbers[72:76], "start": self.board_numbers[16], "finish": self.board_numbers[76:80]},
            "yellow": {"home": self.board_numbers[88:92], "start": self.board_numbers[48], "finish": self.board_numbers[92:96]},
            "red": {"home": self.board_numbers[80:84], "start": self.board_numbers[32], "finish": self.board_numbers[84:88]},
        }
        self.players = ["blue", "green", "yellow", "red"]
        self.player_hands = {player: [] for player in self.players}  # Cards for each player
        self.marble = Marble(pos=0, is_save=False)  # Initialize the marble with valid data (position and is_save)
        
        # Initialize game_state with default values for required fields
        self.game_state = GameState(
            phase=GamePhase.SETUP,
            cnt_round=0,
            bool_game_finished=False,
            bool_card_exchanged=False,
            idx_player_started=0,
            idx_player_active=0,
            list_player=[],
            list_id_card_draw=[],
            list_id_card_discard=[],
            card_active=None
        )  # Set up the game state if necessary

    def get_starting_position(self, color):
        """ Get the starting position for a marble based on its color """
        return self.board[color]["start"]

    def deal_cards(self, LIST_CARD):
        """ Deals cards to each player for five rounds. """
        rounds = range(1, 6)
        cards_per_round = {1: 6, 2: 5, 3: 4, 4: 3, 5: 2}

        for round_number in rounds:
            deal_count = cards_per_round[round_number]
            print(f"Round {round_number}:")
            for player in self.players:
                dealt_cards = random.sample(LIST_CARD, deal_count)
                self.player_hands[player] = dealt_cards
                print(f"  {player}: {self.player_hands[player]}")

    def apply_action(self, action: Action):
        """ Apply the given action to the game. """
        card = action.card
        marble = self.marble  # We use the marble created in the game
        distance = action.pos_to  # the new position to move to
        move_out = action.pos_from  # Used for bringing the marble into play or moving forward
        card.execute(marble, self.board, distance=distance, move_out=move_out)

    def get_player_positions(self, color: str) -> dict:
        """ Get the current positions of a player on the board. """
        return self.board.get(color, None)

    def set_state(self, state: GameState) -> None:
        """ Set the game to a given state """
        pass

    def get_state(self) -> GameState:
        """ Get the complete, unmasked game state """
        return self.game_state

    def print_state(self) -> None:
        """ Print the current game state """
        pass



    def get_list_action(self) -> List[Action]:
        """ Get a list of possible actions for the active player """
        pass

    def get_player_view(self, idx_player: int) -> GameState:
        """ Get the masked state for the active player (e.g. the oppontent's cards are face down)"""
        pass

class RandomPlayer(Player):

    def select_action(self, state: GameState, actions: List[Action]) -> Optional[Action]:
        """ Given masked game state and possible actions, select the next action """
        if len(actions) > 0:
            return random.choice(actions)
        return None


In [64]:
if __name__ == "__main__":
    game = Dog()
    game_state_cards = GameState.LIST_CARD
    game.deal_cards(game_state_cards)

    # Example of using the King, Ace, and Queen cards
    ace_card = Ace()
    # Marble should be instantiated with valid arguments for both pos and is_save
    marble = Marble(pos=0, is_save=False)  # valid initialization


    # Bring marble into play with the King card
    king_card = King()
    king_card.execute(marble, game.board, move_out=True, color="blue")

    # Move marble forward with Ace card
    ace_card.execute(marble, game.board, distance=1)

    # Move marble forward again with Ace card
    ace_card.execute(marble, game.board, distance=11)

    # Move marble forward with King card (13 spaces)
    king_card.execute(marble, game.board, move_out=False)

    # Move marble backward with Queen card
    queen_card = Queen()
    queen_card.execute(marble, game.board)

    print(f"Final position of marble: {marble.pos}")


Round 1:
  blue: [Card(suit='♥', rank='3'), Card(suit='♥', rank='K'), Card(suit='♠', rank='K'), Card(suit='♦', rank='2'), Card(suit='♣', rank='10'), Card(suit='♥', rank='A')]
  green: [Card(suit='♥', rank='6'), Card(suit='♣', rank='9'), Card(suit='♦', rank='J'), Card(suit='♣', rank='3'), Card(suit='♦', rank='A'), Card(suit='♥', rank='10')]
  yellow: [Card(suit='♣', rank='9'), Card(suit='♦', rank='8'), Card(suit='♣', rank='J'), Card(suit='♣', rank='4'), Card(suit='', rank='JKR'), Card(suit='♦', rank='3')]
  red: [Card(suit='♦', rank='A'), Card(suit='♠', rank='10'), Card(suit='♦', rank='5'), Card(suit='♥', rank='4'), Card(suit='♣', rank='7'), Card(suit='♣', rank='J')]
Round 2:
  blue: [Card(suit='♠', rank='A'), Card(suit='♠', rank='6'), Card(suit='♥', rank='K'), Card(suit='♦', rank='A'), Card(suit='♠', rank='5')]
  green: [Card(suit='♥', rank='4'), Card(suit='♦', rank='5'), Card(suit='♦', rank='10'), Card(suit='♥', rank='Q'), Card(suit='♠', rank='8')]
  yellow: [Card(suit='♥', rank='8'),

AttributeError: 'dict' object has no attribute 'get_starting_position'